In [34]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr


In [ ]:

load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

gpt_model = "gpt-4o-mini"
client = OpenAI()

In [36]:
system_message = 'you are helpful assistant for airline called flieghtAI'
system_message += 'give short courtouse answer, no more than 1 sentence'
system_message += "always be accurate If you don't know the answer, say so"


In [37]:
def chat(message, history):
    messages = [{'role': 'system', 'content' : system_message}]
    for human, assistant in history:
        messages.append({'role':'user', 'content': human})
        messages.append({'role':'assistant', 'content': assistant})

    messages.append({'role':'user', 'content': message})

    response = client.chat.completions.create(
        model = gpt_model,
        messages = messages
        )



    return response.choices[0].message.content

In [38]:
gr.ChatInterface(fn = chat).launch()

/opt/anaconda3/envs/udemy_llm/lib/python3.13/site-packages/gradio/chat_interface.py:338: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


* Running on local URL:  http://127.0.0.1:7870
* To create a public link, set `share=True` in `launch()`.


In [ ]:
ticket_prices = {'london': '$799', "paris": "$899", "tokyo": "$1399", 'beijing': '$1599'}
# 챗봇이 조회할 수 있는 정보가 저장된 간단한 데이터베이스 역할을 합니다.
# 실제 어플리케이션에서는 이부분이 DB쿼리 외부 API 호출등으로 대체 될 수 있음

def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}")

    city = destination_city.lower()

    return ticket_prices.get(city, 'unknown')  
    # ticket_prices에서 city인자를 키로 가져오면 가격이 'unknown'자리로 반환 함
    # 예를 들어 딕셔너리 안에 'london'을 호출하면 '$799'가 호출되고 딕셔너리안에 없는 newyork을 호출하면 기본 값이 'unknown'이 호출됨


In [53]:
get_ticket_price('paris')

Tool get_ticket_price called for paris


'$899'

In [54]:
# there's a particular dictionary structure that's required to describe our function:

price_function = {
    "name":"get_ticket_price",
    "description": "Get the ticket price for a destination city, Call this whenever you need to know the ticket price, for example when a customer asks 'how much is the ticket to the city",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            }
        },
        "required": ["destination_city"],
        "additionalProperties":False
    }
}

"""
### OpenAI 함수(도구) 정의 템플릿 상세 설명

| 필드 (계층 포함) | 필수 여부 | 설명 |
| :--- | :--- | :--- |
| `name` | **필수** | 호출될 실제 파이썬 함수의 이름입니다. AI가 이 함수를 호출하기로 결정하면 이 이름을 반환합니다. |
| `description` | 권장 (사실상 필수) | **가장 중요한 필드입니다.** AI가 사용자의 말을 듣고 **언제 이 함수를 호출할지 결정하는 핵심 기준**이 됩니다. 명확하고 상세할수록 AI가 더 똑똑하게 작동합니다. |
| `parameters` | **필수** | 함수에 전달될 인자(파라미터)들을 정의하는 객체입니다. (JSON Schema 형식) |
| ↳ `type` | **필수** | 파라미터 전체를 감싸는 객체의 타입입니다. 일반적으로 항상 `"object"`를 사용합니다. |
| ↳ `properties` | **필수** | 실제 파라미터들을 정의하는 곳입니다. 각 파라미터의 이름이 키(key)가 됩니다. |
| ↳↳ `type` | **필수** | 해당 파라미터의 데이터 타입입니다. (`"string"`, `"number"`, `"integer"`, `"boolean"`, `"array"`, `"object"` 등) |
| ↳↳ `description` | 권장 (사실상 필수) | 해당 파라미터에 대한 설명입니다. AI가 사용자의 문장에서 어떤 정보를 추출하여 이 파라미터에 넣을지 결정하는 데 도움을 줍니다. |
| ↳↳ `enum` | 선택 사항 | 만약 파라미터의 값이 **정해진 선택지 중 하나**여야 한다면, 여기에 리스트 형태로 선택지를 제공할 수 있습니다. (예: 좌석 등급, 카테고리 등) |
| ↳ `required` | **필수** | `properties`에 정의된 파라미터들 중에서 **반드시 필요한 파라미터들의 이름**을 리스트(`list`) 형태로 지정합니다. 이 목록에 있는 정보가 없으면 AI가 함수 호출을 시도하지 않습니다. |
"""

In [55]:
tools = [{'type': "function","function": price_function}]

In [ ]:
import json

# ----------------------------------------------------
# 1. 모든 필드를 사용한 함수(도구) 명세서 정의
# ----------------------------------------------------
pizza_order_tool = {
    "type": "function",
    "function": {
        "name": "order_pizza",
        "description": "피자 주문을 처리합니다. 고객 이름, 피자 사이즈, 수량, 배달 여부를 지정해야 합니다. 토핑과 결제 수단은 선택적으로 지정할 수 있습니다.",
        "parameters": {
            "type": "object",
            "properties": {
                "customer_name": {
                    "type": "string",
                    "description": "주문하는 고객의 이름"
                },
                "size": {
                    "type": "string",
                    "description": "주문할 피자의 사이즈",
                    "enum": ["small", "medium", "large", "x-large"]
                },
                "quantity": {
                    "type": "integer",
                    "description": "주문할 피자의 수량"
                },
                "toppings": {
                    "type": "array",
                    "items": {
                        "type": "string"
                    },
                    "description": "추가하고 싶은 토핑의 목록 (예: ['페퍼로니', '치즈', '올리브'])"
                },
                "is_delivery": {
                    "type": "boolean",
                    "description": "배달 여부를 지정합니다 (true는 배달, false는 매장 픽업)"
                },
                "payment_method": {
                    "type": "string",
                    "description": "결제 수단 (기본값: 'credit_card')",
                    "enum": ["credit_card", "cash"]
                }
            },
            "required": ["customer_name", "size", "quantity", "is_delivery"]
        }
    }
}


# ----------------------------------------------------
# 2. 위 명세서에 해당하는 실제 파이썬 함수
# ----------------------------------------------------
def order_pizza(customer_name: str, 
                size: str, 
                quantity: int, 
                is_delivery: bool, 
                toppings: list = None, 
                payment_method: str = "credit_card"):
    """
    OpenAI 모델이 호출할 실제 피자 주문 처리 함수입니다.
    선택적 파라미터는 기본값을 설정합니다.
    """
    if toppings is None:
        toppings = [] # 토핑이 제공되지 않으면 빈 리스트로 처리

    order_summary = {
        "고객명": customer_name,
        "사이즈": size,
        "수량": quantity,
        "토핑": toppings if toppings else "없음",
        "배달여부": "배달" if is_delivery else "매장 픽업",
        "결제수단": payment_method
    }
    
    print("--- 새로운 주문이 접수되었습니다 ---")
    print(json.dumps(order_summary, indent=2, ensure_ascii=False))
    print("---------------------------------")
    
    return json.dumps(order_summary)

# 이 예시는 함수 정의이므로, 실제 챗봇 로직에서 이 함수와 명세서를 사용해야 합니다.

In [59]:
def chat(message, history):
    messages = [{'role': 'system', 'content' : system_message}]
    for human, assistant in history:
        messages.append({'role':'user', 'content': human})
        messages.append({'role':'assistant', 'content': assistant})
    
    messages.append({'role':'user', 'content': message})

    response = client.chat.completions.create(
        model = gpt_model,
        messages = messages,
        tools = tools # make use tools(price_functions)
    )

    if response.choices[0].finish_reason =="tool_calls":
        message = response.choices[0].message
        response, city = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        response = client.chat.completions.create(
            model = gpt_model,
            messages = messages
        )

    return response.choices[0].message.content

In [60]:
# we have to write that fucntion handle_tool_call:

def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)
    city = arguments.get('destination_city')
    price = get_ticket_price(city)
    response = {
        "role": "tool",
        "content":json.dumps({"destination_city": city, "price": price}),
        "tool_call_id" : message.tool_calls[0].id
    }
    return response, city

In [61]:
gr.ChatInterface(fn=chat).launch()

/opt/anaconda3/envs/udemy_llm/lib/python3.13/site-packages/gradio/chat_interface.py:338: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


* Running on local URL:  http://127.0.0.1:7873
* To create a public link, set `share=True` in `launch()`.


Tool get_ticket_price called for Paris
Tool get_ticket_price called for Tokyo
